In [1]:
import pandas as pd
import numpy as np

# Datos

Los datos serán almacenados en memoria en 5 Data Frames distintos, los cuales contendrán la información vinculada a:

- Personas: Id, Fecha de Nacimiento, Género, Código Postal.
- Usuarios (Personas): Id(Personas), Ocupación, Fecha de Alta.
- Trabajadores (Personas): Id(Personas), Fecha de Alta, Puesto, Categoría, Horario de Trabajo.  
- Películas: Nombre, Año, Género(s), id.  
- Scores: Usuario (Id), Película (Id), Puntuación, Timestamp

In [2]:
personas_db = pd.read_csv("../Data/personas.csv")
personas_db.head()

,id,Full Name,year of birth,Gender,Zip Code
0,1,Robert Stanley,1974,M,85711
1,2,Heather Bossey,1945,F,94043
2,3,Don Davis,1975,M,32067
3,4,Daniel Mccauley,1974,M,43537
4,5,Janice Mccullough,1965,F,15213


In [3]:
usuarios_db = pd.read_csv("../Data/usuarios.csv")
usuarios_db.head()

,id,Occupation,Active Since
0,1,technician,1997-09-22 21:57:58
1,2,other,1998-02-27 03:26:00
2,3,writer,1998-03-07 02:15:39
3,4,technician,1998-04-08 02:09:34
4,5,other,1997-09-30 15:57:44


In [4]:
trabajadores_db = pd.read_csv("../Data/trabajadores.csv")
trabajadores_db.head()

,id,Position,Category,Working Hours,Start Date
0,16,Analyst,B,9 - 18,1998-03-30
1,31,Analyst,A,9 - 18,1997-09-25
2,167,Analyst,B,7 - 16,1997-10-24
3,183,Analyst,C,20 - 04,1998-01-26
4,298,Analyst,C,9 - 18,1997-10-18


In [6]:
scores_db = pd.read_csv("../Data/scores.csv")
scores_db = scores_db.drop(columns="Unnamed: 0")
scores_db.head()

,user_id,movie_id,rating,Date
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


# CLASE PELICULA


In [124]:
def parse_movie(pelicula, df_movies):
    new_row = [pelicula.id,pelicula.nombre,pelicula.anio,"unkown URL"]
    if len(pelicula.generos) == 0: new_row.append(1)
    else: new_row.append(0)
    for gen in df_movies.columns.tolist()[5:]:
        if gen in pelicula.generos: new_row.append(1)
        else: new_row.append(0)
    return new_row

In [4]:
def get_movies_structure:
    return ["id", "Name", "Release Date", "IMDB URL", "unknown", "Action","Adventure","Animation", 
                                "Children's", "Comedy", "Crime","Documentary", "Drama", "Fantasy", "Film-Noir",
                                "Horror", "Musical","Mystery", "Romance","Sci-Fi", "Thriller", "War", "Western"]

SyntaxError: expected '(' (696879055.py, line 1)

In [10]:
class Pelicula:
    def __init__(self, nombre, anio, generos, id = None):
        self.nombre = nombre
        self.anio = anio
        self.generos = generos
        self.id = id

    def __repr__(self):
        # Este método debe imprimir la información de esta película.
        output = f"Nombre: {self.nombre}\nAño: {self.anio}\nGenero: {self.generos}"
        return output
    

    def write_df(self, df_mov):
        # Este método recibe el dataframe de películas y agrega la película
        # Si el id es None, toma el id más alto del DF y le suma uno. Si el 
        # id ya existe, no la agrega y devuelve un error.

        # manejo del id
        if self.id in df_mov["id"].values:
            return -1        
        if self.id is None:
            self.id = df_mov["id"].max() + 1 if not df_mov.empty else 1  

        # escritura en df
        new_row = parse_movie(self, df_mov)      #todo/ validar posible error
        df_mov.loc[len(df_mov)] = new_row   

        return df_mov

    @classmethod
    def create_df_from_csv(cls, filename):
        # Este class method recibe el nombre de un archivo csv, valida su 
        # estructura y devuelve un DataFrame con la información cargada del
        # archivo 'filename'.
        
        try: # Intentamos cargar el archivo CSV en un DataFrame
            df = pd.read_csv(filename)
                  
            if (df.columns !=  get_movies_structure()):
                raise ValueError("El archivo CSV no contiene todas las columnas requeridas.")
            # Si todas las validaciones son exitosas, retornamos el DataFrame
            return df
        
        except Exception as e:
            # Capturamos cualquier excepción que ocurra durante el proceso
            print(f"Error al procesar el archivo CSV '{filename}': {str(e)}")
            return None

    @classmethod
    def get_from_df(cls, df_mov, id=None, nombre=None, anios=None, generos=None):
        # Este class method devuelve una lista de objetos 'Pelicula' buscando por:
            # id: id
            # nombre: nombre de la película
            # anios: [desde_año, hasta_año]
            # generos: [generos]

        peliculas = []

        # Filtrar el DataFrame según los criterios de búsqueda
        mask = df_mov.copy()  # Creamos una copia del DataFrame para evitar modificaciones no deseadas
        if id is not None:
            mask = mask[mask['id'] == id]
        if nombre is not None:
            mask = mask[mask['nombre'].str.contains(nombre, case=False)]
        if anios is not None:
            mask = mask[(mask['anio'] >= anios[0]) & (mask['anio'] <= anios[1])]
        if generos is not None:
            # Iterar sobre los géneros y aplicar el filtro
            for genero in generos:
                mask = mask[mask[f'{genero}'] == 1]
        
        # Iterar sobre las filas filtradas y crear objetos Pelicula
        for index, row in mask.iterrows():
            pelicula = cls(row['id'], row['nombre'], row['anio'], row['genero'])
            peliculas.append(pelicula)
        
        return peliculas

    @classmethod
    def get_stats(cls, df_mov, anios=None, generos=None):
        # Este class method imprime una serie de estadísticas calculadas sobre
        # los resultados de una consulta al DataFrame df_mov. 
        # Las estadísticas se realizarán sobre las filas que cumplan con los requisitos de:
        # anios: [desde_año, hasta_año]
        # generos: [generos]
        # Las estadísticas son:
        # - Datos película más vieja
        # - Datos película más nueva
        # - Bar plots con la cantidad de películas por año/género.

        # Filtrar por años
        if anios:
            desde_año, hasta_año = anios
            df_mov = df_mov[(df_mov['año'] >= desde_año) & (df_mov['año'] <= hasta_año)]
        
        # Filtrar por géneros
        if generos:
            df_mov = df_mov[df_mov['género'].isin(generos)]
        
        # Película más vieja
        pelicula_mas_vieja = df_mov.loc[df_mov['año'].idxmin()]
        print("Película más vieja:")
        print(pelicula_mas_vieja)
        
        # Película más nueva
        pelicula_mas_nueva = df_mov.loc[df_mov['año'].idxmax()]
        print("\nPelícula más nueva:")
        print(pelicula_mas_nueva)
        
        # Gráfico de barras de la cantidad de películas por año
        plt.figure(figsize=(10, 6))
        df_mov['año'].value_counts().sort_index().plot(kind='bar')
        plt.title('Cantidad de películas por año')
        plt.xlabel('Año')
        plt.ylabel('Cantidad de películas')
        plt.show()
        
        # Gráfico de barras de la cantidad de películas por género
        plt.figure(figsize=(10, 6))
        df_mov['género'].value_counts().plot(kind='bar')
        plt.title('Cantidad de películas por género')
        plt.xlabel('Género')
        plt.ylabel('Cantidad de películas')
        plt.show()

    def remove_from_df(self, df_mov):
        1
    # Borra del DataFrame el objeto contenido en esta clase.
    # Para realizar el borrado todas las propiedades del objeto deben coincidir
    # con la entrada en el DF. Caso contrario imprime un error.


peli = Pelicula("Rocky VII", 2010, ["Action","Drama","Thriller"])

In [1]:
peliculas_db = Pelicula.create_df_from_csv("../Data/peliculas.csv")
pelis_short = peliculas_db.head(3)
peli.write_df(pelis_short)


In [194]:
peliculas_db["Release Date"].iloc[10:100]

10    01-Jan-1995
11    14-Aug-1995
12    30-Oct-1995
13    01-Jan-1994
14    29-Jan-1996
         ...     
95    01-Jan-1991
96    01-Jan-1990
97    01-Jan-1991
98    01-Jan-1937
99    14-Feb-1997
Name: Release Date, Length: 90, dtype: object

In [ ]:
""